<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_9_tuning_healthnews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    989      0 --:--:-- --:--:-- --:--:--   989
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-9.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [12]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-9.validation.csv",header=None)
valid.head()

,0,1
0,3,cracking sensations in my joints
1,5,fast and slow heartbeat
2,8,arthritis
3,8,arthritis
4,8,arthritis


In [13]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-9.test.csv",header=None)
test.head()

,0,1
0,2,painful heels
1,4,Discoloration of skin [blotchy] on the neck
2,5,my heart rate jumped to 106
3,7,goiter
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_healthnews_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [18]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [23]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,6.959872,5.772547,0.144810,02:05


In [0]:
# learn.fit_one_cycle(1, 1e-2)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
learn.save_encoder('healthnews_fold9_first')

In [28]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('healthnews_fold9_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.619326,4.143027,0.343666,01:06


In [29]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.935381,3.880640,0.384097,01:18


In [30]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.425694,3.655421,0.402965,03:16


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.309567,3.493435,0.409703,03:15
1,3.101760,3.344980,0.431267,03:35


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.057329,3.251214,0.450135,03:19
1,2.868655,3.214150,0.460916,03:16


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.827678,3.224370,0.467655,03:09
1,2.686048,3.102301,0.473046,03:31


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.748164,3.045703,0.485175,03:33
1,2.590684,3.008456,0.485175,03:13


In [35]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.626065,3.031184,0.482480,03:30
1,2.484735,2.988201,0.498652,03:17


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.480919,3.027075,0.486523,03:17
1,2.497958,2.985608,0.489218,03:15
2,2.365537,2.939006,0.491914,03:18
3,2.311106,2.928778,0.493261,03:17


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.281817,2.969498,0.500000,03:09
1,2.376374,2.940513,0.512129,03:13
2,2.259053,2.886491,0.512129,03:27
3,2.222587,2.926550,0.509434,03:08


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.183750,2.910001,0.509434,03:28
1,2.229118,2.905430,0.510782,03:18
2,2.190906,2.881091,0.513477,03:29
3,2.172380,2.878499,0.521563,03:15


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.195592,2.903328,0.513477,03:28
1,2.221156,2.905561,0.512129,03:21
2,2.144125,2.902026,0.521563,03:14
3,2.061378,2.892405,0.514825,03:26


In [0]:
# save the best model

learn.save_encoder('ask-9-tuning-healthnews')

In [44]:
learn.load_encoder('ask-9-tuning-healthnews')
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.162736,2.938907,0.512129,01:10
1,2.091159,2.896284,0.517520,01:09
2,2.180085,3.025674,0.508086,01:09
3,2.120502,2.960393,0.514825,01:04


In [45]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.082895,2.941978,0.509434,01:10
1,2.108960,2.940511,0.510782,01:12
2,2.129540,2.911814,0.514825,01:05
3,2.076655,2.875382,0.512129,01:07


In [46]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.047272,2.930514,0.510782,01:04
1,2.100281,2.871320,0.516173,01:04
2,2.061448,2.926460,0.512129,01:04
3,2.028634,2.966471,0.514825,01:04


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

In [47]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.082244,2.916599,0.514825,01:08
1,2.080476,2.933384,0.512129,01:10
2,2.081399,2.927134,0.514825,01:10
3,2.121197,3.037157,0.512129,01:03


In [48]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.013596,2.946426,0.517520,01:11
1,2.139206,2.896950,0.516173,01:10
2,2.085215,2.935607,0.512129,01:06
3,2.119589,3.004196,0.508086,01:09


# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [50]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[228, 841, 18, 94, 8, 8, 8, 8, 8, 164, 16, 16, 824, 20, 214, 329, 536, 197, 27, 749, 31, 202, 584, 289, 741, 37, 37, 896, 883, 805, 289, 289, 363, 877, 1026, 606, 1026, 49, 49, 49, 711, 363, 711, 711, 517, 1023, 1035, 658, 1035, 50, 126, 890, 289, 289, 567, 228, 711, 711, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 265, 172, 711, 711, 711, 711, 289, 711, 289, 80, 877, 491, 49, 85, 660, 88, 89, 137, 214, 214, 214, 92, 711, 711, 711, 711, 711, 711, 711, 711, 318, 234, 711, 716, 737, 735, 511, 100, 100, 214, 100, 100, 108, 108, 108, 660, 111, 741, 604, 111, 117, 896, 117, 1035, 125, 711, 711, 711, 711, 125, 125, 125, 125, 125, 125, 125, 125, 125, 343, 129, 343, 1026, 225, 268, 268, 137, 473, 140, 447, 255, 711, 711, 158, 711, 711, 321, 711, 711, 711, 711, 711, 711, 318, 76, 604, 2, 704, 364, 289, 289, 289, 753, 711, 427, 165, 711, 245, 182, 289, 289, 1035, 183, 183, 183, 183, 183, 183, 711, 187, 187, 

In [51]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

238
0.2748267898383372
